# AVI Python SDK. First Steps

AVI API SDK is a Python Package that provides APIs to communicate with Avi Controller’s REST APIs. It extends Python’s Request Library’s Session Class and provides utilities to simplify integration with Avi Controller.

https://avinetworks.com/docs/latest/api-guide/overview.html 


https://github.com/vmware/alb-sdk

- [1.- Initial Login](#1--initial-login)
- [2.- Read Operations](#2--read-operations)
- [3.- Working with JSON](#3--working-with-data-structures)

## 1.- Initial Login

The Session Class handles session authentication and keeps a cache of sessions to avoid multiple connection setups and teardowns across different API Session invocation. It automatically updates session cookies, CSRF Tokens from controller and provides helper APIs and templates for Avi Objects. Other important features are X-AVI-TENANT (tenant) header handling and sample source code for common load balancing examples


### Import Libraries

In [ ]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json

### Import Environment Session Variables

In this example, the session parameters such a username, password, Controller IP Address and so on have been provided from an independet file that is placed in envs/controller_info.py file. From this file we are importing the session_params data as session_env and we can access easily to the values of all the existing variables as shown bellow. 

In [ ]:
from envs.controller_info import session_params as session_env

In [ ]:
# Establish a first session
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )

### Display Session ID to Verify AVI Controller Session Establishment

In [ ]:
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

From this moment on, we can call the **api** object and invoke different methods that provide CRUD operations as in the following examples. 

## 2.- Read (GET) Operations

Get is a common method to read information.
You can get information of your current setup easily invoking the corresponding API

AVI has dozens of APIs... 
   - Use https://controller/swagger to understand different API
   - You can easily infere the API when navigating through the GUI just looking at the address bar used in the GUI 

   Example if the GUI shows https://192.168.1.15/#!/admin/applications/virtualservice that means you are in the **admin** tenant and you are using the **virtualservice** API

   In the previous step we created an AVI API session object that we called *api*. 
   Let's invoke the **GET** method over our api object to inspect the output

### 2.1 API Object **Get** Method

Now create a new object named containing the response received from the controller. The response object contains some properties such as text or url that we can read after sending the GET request. In this case the api we are using is **virtualservice**

In [ ]:
resp = api.get('virtualservice')
print ("Sending request to following URL: " + resp.url)

The request produces a json formatted output. You can beautify the output it by using json loads method to gain some readability

In [ ]:
json.loads(resp.text)

As you can tell, above output can be hard to handle and read. There are some parameters you can use to narrow down the response.

- **path**: takes relative path to the AVI api 
- **tenant**: overrides the tenant used during session creation
- **tenant_uuid**: overrides the tenant or tenant_uuid during session creation  
- **timeout**: timeout for API calls; Default value is 60 seconds 
- **api_version**: overrides x-avi-header in request header during session creation get method takes relative path to service and kwargs as per Session class get method returns session's response object
- **params**: dictionary of key value pairs to be sent as query parameters


| Filter                                               | Description                                                                   | Example                                                                                                                                                                                                    | Returns                                                                                                                                                         |
| ---------------------------------------------------- | ----------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| \<field\>=                                             | Filter objects with specific fields matching given value                      | /api/pool/?fail_action.type=FAIL_ACTION_CLOSE_CONN                                                                                                                                                         | Returns list of pools whose failure action is configured as FAIL_ACTION_CLOSE_CONN                                                                              |
|                                                      |\<field\> can be a nested field using . as a separator | /api/pool/?analytics_policy.enable_realtime_metrics=false                     | Returns list of pools with real-time metrics enabled                                                                                                                                                       |
| \<field\>.contains=                                    | Filter objects where field \<field\> contains the given string                  | /api/pool/?name.contains=-app1-                                                                                                                                                                            | Returns list of pools whose name contains the string "-app1-"                                                                                                   |
| \<field\>.icontains=                                   | ("contains" is case-sensitive, "icontains" is case-insensitive)               |
| [name.in](http://name.in)                            | Filter objects where name or uuid exactly matches any of the values in a list | /api/pool/?name.in=pool-1,pool-2,pool-3                                                                                                                                                                    | Returns list with pool-1, pool-2 and pool-3                                                                                                                     |
| [uuid.in](http://uuid.in)                            |
| search                                               | Filter objects which contain the search string in any field’s value           | /api/pool/?search=LEAST_CONN                                                                                                                                                                               | Returns list of pools containing "LEAST_CONN" in any field value. To search for an exact match on an entire field’s value, enclose the search string in quotes. |
| isearch                                              | /api/pool/?search="LB_ALGORITHM_LEAST_CONNECTIONS"                            | ("search" is case-sensitive, "isearch" is case-insensitive)                                                                                                                                                |
| search=()                                            | Filter objects by searching for a field name and value                        | /api/pool/?search=(addr,10.1.1.1)                                                                                                                                                                          | Returns list of pools containing the string "10.1.1.1" in the field "addr" (even if the field is a nested field as in this case).                               |
| isearch=()                                           | /api/pool/?search=(addr, "10.1.1.1")                                          | To search for an exact match on an entire field’s value, enclose the search string in quotes. Leaving the value part empty matches any value (i.e. searches for the presence of the specified field name). |
|                                                      | /api/virtualservice/?search=(dns_policies,)                                   | ("search" is case-sensitive, "isearch" is case-insensitive)                                                                                                                                                |
| refers_to                                            | Filter objects which have a reference to the specified object                 | /api/pool/?refers_to=healthmonitor:healthmonitor-be7880b9-2c72-4abd-9fd3-8243517e8163                                                                                                                      | Returns list of pools that refer to the health monitor with this UUID                                                                                           |
| referred_by                                          | Filter objects which are referred to by the specified object                  | /api/pool/?referred_by=virtualservice:virtualservice-7a292eb3-ad23-4900-a269-c529db4133f8                                                                                                                  | Returns list of pools that are referred to by the virtual service with this UUID                                                                                |
| exclude                                              | Inverts the filtering logic of the specified query                            | /api/pool/?name.icontains=-web-&refers_to=healthmonitor:healthmonitor-be7880b9-2c72-4abd-9fd3-8243517e8163&exclude=name.icontains                                                                          | Returns list of pools that refer to the health monitor with this UUID where the Pool name does not contain the string "-web-"                                   |
| skip_default                                         | Excludes fields from the returned object that are set to the default value    | /api/pool/?name=app1-pool&skip_default=true                                                                                                                                                                | Returns the specified object excluding any field whose value has not been changed from its default                                                              |
| include_name                                         | Appends the object name to each object reference - \<object\>_ref: \<url\>#\<name\> | /api/pool/?name=app1-pool&include_name=true                                                                                                                                                                | Returns the object with all \*_ref fields having the referenced object name appended                                                                            |

**Example 1**.- Get virtualservices with a name value equals to **vs-example-org1**

_Sample output:_

```json
{'count': 1,
 'results': [{'_last_modified': '1720707512296814',
   'active_standby_se_tag': 'ACTIVE_STANDBY_SE_1',
   'advertise_down_vs': False,
   'allow_invalid_client_cert': False,
   ... # (skipped)
   'close_client_conn_on_config_update': False,
   'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
   'cloud_type': 'CLOUD_NONE',
   'content_rewrite': {'rewritable_content_ref': 'https://192.168.1.15/api/stringgroup/stringgroup-caeef771-f17f-4099-936b-33023eb37300'},
   'delay_fairness': False,
   'east_west_placement': False,
   'weight': 1}]}
```

In [14]:
query= { 
    "name": "vs-example-org1"
    }
resp = api.get('virtualservice', params=query)
print ("Sending request to following URL: " +resp.url)
json.loads(resp.text)

Sending request to following URL: https://192.168.1.15/api/virtualservice?name=vs-example-org1


{'count': 1,
 'results': [{'_last_modified': '1724087488095927',
   'active_standby_se_tag': 'ACTIVE_STANDBY_SE_1',
   'advertise_down_vs': False,
   'allow_invalid_client_cert': False,
   'analytics_policy': {'all_headers': False,
    'client_insights': 'NO_INSIGHTS',
    'full_client_logs': {'duration': 30, 'enabled': False, 'throttle': 10},
    'metrics_realtime_update': {'duration': 30, 'enabled': False},
    'significant_log_throttle': 10,
    'udf_log_throttle': 10},
   'analytics_profile_ref': 'https://192.168.1.15/api/analyticsprofile/analyticsprofile-768ef0ed-a2ac-427c-92c7-8715ce805374',
   'application_profile_ref': 'https://192.168.1.15/api/applicationprofile/applicationprofile-d6f57996-7ee0-4993-8eee-d95f82c84c45',
   'bulk_sync_kvcache': False,
   'close_client_conn_on_config_update': False,
   'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
   'cloud_type': 'CLOUD_NONE',
   'delay_fairness': False,
   'east_west_placement': False

**Example 2**.- Get virtualservices with a name value equal to vs-example-org1 but only retrieve some fields in the response (enabled, vsvip_ref and type).

_Sample output_:

```json
{'count': 1,
 'results': [
    {'enabled': True,
   'name': 'vs-example-org1',
   'type': 'VS_TYPE_NORMAL',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'vsvip_ref': 'https://192.168.1.15/api/vsvip/vsvip-188553aa-bbe1-4b6e-b2ed-7ed1d5f78293'}
  ]
}
```

In [15]:
query= { 
    "name": "vs-example-org1", 
    "fields": "enabled,vsvip_ref,type"
    }
resp = api.get('virtualservice', params=query)
print ("Sending request to following URL: " + resp.url)
json.loads(resp.text)

Sending request to following URL: https://192.168.1.15/api/virtualservice?name=vs-example-org1&fields=enabled%2Cvsvip_ref%2Ctype


{'count': 1,
 'results': [{'enabled': True,
   'name': 'vs-example-org1',
   'type': 'VS_TYPE_NORMAL',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23',
   'uuid': 'virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23',
   'vsvip_ref': 'https://192.168.1.15/api/vsvip/vsvip-b2dc6a93-4bc1-4462-a778-b22dbf0a6e9b'}]}

**Example 3**.- Get virtualservices that contains the string **-Example-** in the field **name** (key insensitive) and only retrieve some fields in the response (name).

_Sample output_:

```json
{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example-org1-new-name',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646',
   'uuid': 'virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646'}]}
```

In [16]:
query= { 
    "name.icontains": "-Example-", 
    "fields": "name"
    }
resp = api.get('virtualservice', params=query)
print ("Sending request to following URL: " + resp.url)
json.loads(resp.text)

Sending request to following URL: https://192.168.1.15/api/virtualservice?name.icontains=-Example-&fields=name


{'count': 1,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23',
   'uuid': 'virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23'}]}

**Example 4**.- Using two queries obtain the virtualservices sharing the same service engine group with a given virtualservice named **vs-example-org1**.
    
  - 1st query get se_group_ref for a given virtualservice name  

_Sample output_first_query:_

```json
{'count': 1,
 'results': [{'name': 'vs-example-org1',
   'se_group_ref': 'https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-17a49de4-afad-4929-a8e1-297be6ab9d70#SOURCE-GROUP1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c#vs-example-org1',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'}]}
```
  - 2nd query after extracting the UUID (embedded in the se_group_ref retrieved field above) use the refers_to query parameter to get all virtualservice sharing same service engine group

_Sample output_second_query:_

```json
{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example2-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076',
   'uuid': 'virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076'}]}
```


In [17]:
# 1st query retrieve se_group_ref appending the name to ease identification
query= {
    "name": "vs-example-org1", 
    "fields": "name,se_group_ref", # Without blank spaces
    "include_name": "true"
    }
resp = api.get('virtualservice', params=query)
print ("Sending request to following URL: " + resp.url)
json.loads(resp.text)

Sending request to following URL: https://192.168.1.15/api/virtualservice?name=vs-example-org1&fields=name%2Cse_group_ref&include_name=true


{'count': 1,
 'results': [{'name': 'vs-example-org1',
   'se_group_ref': 'https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f#SEG-TARGET-MAD-001',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23#vs-example-org1',
   'uuid': 'virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23'}]}

Extract the se_group_ref (removing the trailing name delimited by #)

In [19]:
# Copy response text to the output variable
output=json.loads(resp.text)
# Access the first (0 index) of results and extract the se_group_ref field splitting the trailing name delimited by #. Store in se_group_ref_name variable
se_group_ref_name = output["results"][0]["se_group_ref"].split("#")[0]
# Extract the SE_GROUP_UUID from the se_group_ref (last part of URL)
se_group_uuid = se_group_ref_name.split("/")[5]
print("The extracted Service Engine Group UUID is "+se_group_uuid)

The extracted Service Engine Group UUID is serviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f


In [20]:
# 2st query retrieve all virtual services that refer to the service Engine Group UUID by that se_group_ref
query= {
    "fields": "name", # Without blank spaces
    "refers_to": f"serviceenginegroup:{se_group_uuid}"
    }
resp = api.get('virtualservice', params=query)
print ("Sending request to following URL: " + resp.url)
json.loads(resp.text)

Sending request to following URL: https://192.168.1.15/api/virtualservice?fields=name&refers_to=serviceenginegroup%3Aserviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f


{'count': 7,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23',
   'uuid': 'virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23'},
  {'name': 'VS_SOURCE_001',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-d29b49d5-b3a0-47ed-bfd2-f6462ff73e14',
   'uuid': 'virtualservice-d29b49d5-b3a0-47ed-bfd2-f6462ff73e14'},
  {'name': 'VS_SOURCE_002',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-6aafed40-0c8e-4e47-b970-b9eefa8464aa',
   'uuid': 'virtualservice-6aafed40-0c8e-4e47-b970-b9eefa8464aa'},
  {'name': 'VS_SOURCE_003',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-77e6a8c1-e71f-4cd9-aadf-6b07bc8857a3',
   'uuid': 'virtualservice-77e6a8c1-e71f-4cd9-aadf-6b07bc8857a3'},
  {'name': 'VS_TARGET_001',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-ed765748-3625-4bd4-bb24-452152d268d9',
   'uuid': 'virtualservice-ed765748-3625-

### 2.2 API GET Object by Name method

Another interesing method is the **get_obj_by_name**. This method is very useful when you know the name of the object you are looking for. Instead of parsing the contents of the results in the full json response and creating a query using the name field, this method directly returns the response of a particular object. 

In [23]:
# Get an specific object by his name 
api_name = "virtualservice"
object_name = "vs-example-org1"
object = api.get_object_by_name(api_name, object_name)
# Display object content
object 

{'_last_modified': '1724087488095927',
 'active_standby_se_tag': 'ACTIVE_STANDBY_SE_1',
 'advertise_down_vs': False,
 'allow_invalid_client_cert': False,
 'analytics_policy': {'all_headers': False,
  'client_insights': 'NO_INSIGHTS',
  'full_client_logs': {'duration': 30, 'enabled': False, 'throttle': 10},
  'metrics_realtime_update': {'duration': 30, 'enabled': False},
  'significant_log_throttle': 10,
  'udf_log_throttle': 10},
 'analytics_profile_ref': 'https://192.168.1.15/api/analyticsprofile/analyticsprofile-768ef0ed-a2ac-427c-92c7-8715ce805374',
 'application_profile_ref': 'https://192.168.1.15/api/applicationprofile/applicationprofile-d6f57996-7ee0-4993-8eee-d95f82c84c45',
 'bulk_sync_kvcache': False,
 'close_client_conn_on_config_update': False,
 'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
 'cloud_type': 'CLOUD_NONE',
 'delay_fairness': False,
 'east_west_placement': False,
 'enable_autogw': False,
 'enabled': True,
 'flow_dist': '

### 2.3 API GET Object Ref method

Similarly the **get_obj_ref** is useful to get the reference value of certain object. This is sometimes required in some of CRUD operations. 

In [30]:
# Get an specific object by his name 
api_name = "serviceenginegroup"
object_name = "SEG-TARGET-MAD-001"
object = api.get_object_by_name(api_name, object_name)

# Get object reference 
object_ref = api.get_obj_ref(object)
print('The '+api_name+' reference of the '+api_name+' named '+object_name+' is:')
print('- '+ str(object_ref))

The serviceenginegroup reference of the serviceenginegroup named SEG-TARGET-MAD-001 is:
- https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f


## 3.- Working with Data Structures

Lets now get familiar with data structures and some tipical operations. As a first step, let's create a new GET virtualservices request and we save the received response into a variable named **data**

In [31]:
query = {
   "skip_default": "true"
}
resp = api.get('virtualservice', params=query)
data = json.loads(resp.text)

print ("Sending request to following URL: " + resp.url)


Sending request to following URL: https://192.168.1.15/api/virtualservice?skip_default=true


Once we have the response, we can save it permanently into a file as json format (double quotes). This can help to explore the contents or even to modify the JSON document for further operations. 

In [32]:
# Save into a file
with open('output.json', 'w') as outfile:
    json.dump(data, outfile)

Similarly we can open an existing JSON file (double quotes) and load the contents into a Python Dictionary (single quotes)

In [33]:
file_path = 'output.json'
with open(file_path, 'r') as inputfile:
    data = json.load(inputfile)
data

{'count': 7,
 'results': [{'_last_modified': '1724087488095927',
   'analytics_policy': {'full_client_logs': {'enabled': False},
    'metrics_realtime_update': {'enabled': False}},
   'analytics_profile_ref': 'https://192.168.1.15/api/analyticsprofile/analyticsprofile-768ef0ed-a2ac-427c-92c7-8715ce805374',
   'application_profile_ref': 'https://192.168.1.15/api/applicationprofile/applicationprofile-d6f57996-7ee0-4993-8eee-d95f82c84c45',
   'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
   'enable_autogw': False,
   'http_policies': [{'http_policy_set_ref': 'https://192.168.1.15/api/httppolicyset/httppolicyset-93309f9a-a77a-4ae3-8295-231fcb34a398',
     'index': 11}],
   'marked_for_delete': False,
   'name': 'vs-example-org1',
   'network_profile_ref': 'https://192.168.1.15/api/networkprofile/networkprofile-4ee2f4b9-728c-4437-a81a-0f7ad3550097',
   'network_security_policy_ref': 'https://192.168.1.15/api/networksecuritypolicy/networksecuritypo

The response contents always include a **count** key that contains a value of the number of occurrences of the requested object whereas the response itself is contained as the value of the **results** key. The results key is essentially a _list of dictionaries_ Python data structure. We can refer the items using the index (0 for the first response, 1 for second response and so on). As an example, let's access to the first item

In [34]:
# Saving results in a new variable and access the first item of the list of dictionaries. 
results = data["results"]
results[0]

{'_last_modified': '1724087488095927',
 'analytics_policy': {'full_client_logs': {'enabled': False},
  'metrics_realtime_update': {'enabled': False}},
 'analytics_profile_ref': 'https://192.168.1.15/api/analyticsprofile/analyticsprofile-768ef0ed-a2ac-427c-92c7-8715ce805374',
 'application_profile_ref': 'https://192.168.1.15/api/applicationprofile/applicationprofile-d6f57996-7ee0-4993-8eee-d95f82c84c45',
 'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
 'enable_autogw': False,
 'http_policies': [{'http_policy_set_ref': 'https://192.168.1.15/api/httppolicyset/httppolicyset-93309f9a-a77a-4ae3-8295-231fcb34a398',
   'index': 11}],
 'marked_for_delete': False,
 'name': 'vs-example-org1',
 'network_profile_ref': 'https://192.168.1.15/api/networkprofile/networkprofile-4ee2f4b9-728c-4437-a81a-0f7ad3550097',
 'network_security_policy_ref': 'https://192.168.1.15/api/networksecuritypolicy/networksecuritypolicy-5bf8a4c3-1929-4163-92aa-0a2759ac6a49',
 'pool

### 3.- Filtering data

Sometimes we want to extract an item (a single dictionary) from the existing list using a value of one of the fields as search element. For example, lets say from the n objects in above example, we want to get only the one where the **name** key contains a value equals to **vs-example-org1**. To implement this, we need to iterate over existing list until a coincidence is found using following sample code: 

In [35]:
print("Number of items in results list: " + str(len(results)))
field_to_match = "name"
value_to_match = "vs-example-org1"
result = [ item for item in results if item.get(field_to_match) == value_to_match]
print("Number of items in new filtered list: " + str(len(result)))

Number of items in results list: 7
Number of items in new filtered list: 1


In [36]:
print("The value of the name key for the single item is: " +str(result[0]["name"]))

The value of the name key for the single item is: vs-example-org1


Another typical operation is when we want to simplify the data by removing some irrelevant fields. As an example imagine we just want to extract the name, uuid, vsvip_ref, se_group_ref, vsvip_ref and pool_ref from the above full response. This can be done through following piece of code. 

_Sample output_:

```json
{'uuid': 'virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23',
 'se_group_ref': 'https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-e5b13687-efbb-4417-9302-b1d37c367c8b',
 'name': 'vs-example-org1',
 'vsvip_ref': 'https://192.168.1.15/api/vsvip/vsvip-b2dc6a93-4bc1-4462-a778-b22dbf0a6e9b',
 'pool_ref': 'https://192.168.1.15/api/pool/pool-66773fdb-8d10-4823-9257-51fffed6b1d4'}
```

In [37]:
# Save data to filter into a new variable
data_to_filter = result[0]

# Define list containing fields_of_interest
fields_of_interest = [ 
    'uuid',
    'se_group_ref',
    'name', 
    'vsvip_ref',
    'pool_ref']

# Filter the data iterating through the items and save results
filtered_data = {key: data_to_filter[key] for key in fields_of_interest if key in data_to_filter }
print("The new filtered data is:")
print(json.dumps(filtered_data, indent=3))

The new filtered data is:
{
   "uuid": "virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23",
   "se_group_ref": "https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f",
   "name": "vs-example-org1",
   "vsvip_ref": "https://192.168.1.15/api/vsvip/vsvip-b2dc6a93-4bc1-4462-a778-b22dbf0a6e9b",
   "pool_ref": "https://192.168.1.15/api/pool/pool-66773fdb-8d10-4823-9257-51fffed6b1d4"
}
